In [21]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import ast
import pandas as pd

In [2]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/Baud/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [3]:
#loading the first page of a search for milk in the web browser
milk = "https://www.heb.com/search/results?N=3139289889&No=0&Nrpp=60&Ntt=milk&q=milk&prodFilter=none"
browser.visit(milk)

In [4]:
#scraping through the first page of results
html = browser.html
soup = bs(html,"lxml")
page = soup.body.find_all('a',id=re.compile('product-'))
milks = page
#had to use find by xpath because one result had the word "Next" in the text
browser.find_by_xpath('//a[@aria-label="go to next page"]').click()
#using a test for whether the next button is present to continue through the rest of the results
while browser.is_element_present_by_xpath('//a[@aria-label="go to next page"]'):
    html = browser.html
    soup = bs(html,"lxml")
    page = soup.body.find_all('a',id=re.compile('product-'))
    
    milks.extend(page)
    try:
        browser.find_by_xpath('//a[@aria-label="go to next page"]').click()
    except:
        break

In [11]:
browser.quit()

In [5]:
milks[120].contents[1]

<div class="responsivegriditem-top" onclick="gtm_e_productClick('gtm-product-click-1283228');">
<div class="cat-feature clearfix">
<div class="cat-feature-list clearfix">
<img src="https://images.heb.com/is/image/HEBGrocery/snp-small/snipe-svegetarian.jpg" title="VEGETARIAN"/>
<img src="https://images.heb.com/is/image/HEBGrocery/snp-small/snipe-sls.jpg" title="LOW SODIUM"/>
<img src="https://images.heb.com/is/image/HEBGrocery/snp-small/snipe-skosher.jpg" title="KOSHER"/>
<img src="https://images.heb.com/is/image/HEBGrocery/snp-small/snipe-svegan.jpg" title="VEGAN"/>
</div></div>
<div class="cat-list-img gtm-product">
<img alt="" class="lazyloaded" data-src="https://images.heb.com/is/image/HEBGrocery/prd-small/so-delicious-dairy-free-original-coconut-milk-001283228.jpg" src="https://images.heb.com/is/image/HEBGrocery/prd-small/so-delicious-dairy-free-original-coconut-milk-001283228.jpg"/><script type="application/ld+json">
{	
	"name": "So Delicious Dairy Free Original Coconut Milk", 
	"

In [14]:
#milks[20]
dictionaries = []
for i in range(len(milks)):
    tag_string = ""
    for item in milks[i].contents[1]:
         tag_string += str(item)

    tag_string = ""
    for item in milks[i].contents[1]:
        tag_string += str(item)

    #tag_string
    item_info = tag_string.split("{")
    the_right_stuff = item_info[1]
    while "\t" in the_right_stuff:
        index = the_right_stuff.find('\t')
        the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+1:]
    while "\n" in the_right_stuff:
        index = the_right_stuff.find('\n')
        the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+1:]
    while "</script>" in the_right_stuff:
        index = the_right_stuff.find('</script>')
        the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+9:]
    while "</div>" in the_right_stuff:
        index = the_right_stuff.find('</div>')
        the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+6:]
    the_right_stuff = '{' + the_right_stuff
    the_right_stuff = ast.literal_eval(the_right_stuff)
    dictionaries.append(the_right_stuff)

In [18]:
dictionaries[30]

{'name': 'H-E-B Original Almond Milk',
 'id': '1435589',
 'category': 'Dairy & Eggs/Milk',
 'brand': 'H-E-B',
 'variant': 'each',
 'price': 2.65,
 'list': 'Search Results page',
 'position': '31',
 'dimension26': 'true',
 'dimension40': 'default'}

In [22]:
milks_df = pd.DataFrame(dictionaries)
milks_df

,name,id,category,brand,variant,price,list,position,dimension26,dimension40
0,Central Market Organics Whole Milk,931316,Dairy & Eggs/Milk,Central Market,each,5.85,Search Results page,1,true,default
1,Central Market Organics Reduced Fat 2% Milkfat...,931317,Dairy & Eggs/Milk,Central Market,each,5.85,Search Results page,2,true,default
2,H-E-B Select Ingredients MooTopia Lactose Free...,671076,Dairy & Eggs/Milk,H-E-B,each,4.10,Search Results page,3,true,default
3,H-E-B Select Ingredients Fat Free Milk,314136,Dairy & Eggs/Milk,H-E-B,each,1.88,Search Results page,4,true,default
4,H-E-B Select Ingredients 1% Low Fat Milk,314134,Dairy & Eggs/Milk,H-E-B,each,1.88,Search Results page,5,true,default
...,...,...,...,...,...,...,...,...,...,...
895,Hill Country Fare 1% Low Fat Milk,314127,Dairy & Eggs/Milk,Hill Country Fare,each,3.07,Search Results page,896,true,default
896,Hill Country Fare Reduced Fat 2% Milk,314131,Dairy & Eggs/Milk,Hill Country Fare,each,3.07,Search Results page,897,true,default
897,Hill Country Fare Whole Milk,314124,Dairy & Eggs/Milk,Hill Country Fare,each,3.07,Search Results page,898,true,default
898,Hill Country Fare Fat Free Milk,314128,Dairy & Eggs/Milk,Hill Country Fare,each,3.07,Search Results page,899,true,default
